# Normalizing Dataset

In [5]:
import sqlite3
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Exception as e:
        print(e)

    return conn

# Function to create table
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Exception as e:
        print(e)

# Function to execute SQL statement
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)
    rows = cur.fetchall()
    return rows

def insert_data(conn, table_name, columns, values):
    cursor = conn.cursor()
    placeholders = ', '.join(['?'] * len(values[0]))  # Create placeholders for each row's values
    insert_sql = f"INSERT OR IGNORE INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"
    cursor.executemany(insert_sql, values)  # Insert all rows in one go
    conn.commit()

# Reading the data, creating Table and inserting the data into Tables

In [47]:
import sqlite3

# Step 1: Parse Dataset into a Dictionary
def parse_dataset(file_path):
    data_dict = {}
    k=0
    with open(file_path, 'r') as f:
        lines = f.readlines()
        column_names = []
        for line in lines:
            line = line.strip()
            if line.startswith("@DATA"):
                k=1
                continue
            elif line.startswith("@"):
                continue
            elif len(column_names) == 0:
                column_names = [
                    "Upper_Age", "Lower_Age", "Reco_Policy_Premium", "City_Code", 
                    "Accomodation_Type", "Reco_Insurance_Type", "Is_Spouse", 
                    "Health_Indicator", "Holding_Policy_Duration", "Holding_Policy_Type", "class"
                ]
                data_dict = {col: [] for col in column_names}
            elif k==1:
                values = line.split(",")
                for col, val in zip(column_names, values):
                    data_dict[col].append(val)
    return data_dict

# Step 2: Define Tables in 3NF
def create_tables(conn):
    tables_sql = [
        """
        CREATE TABLE IF NOT EXISTS City (
            City_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            City_Code TEXT UNIQUE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS HealthIndicator (
            Indicator_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Health_Indicator TEXT UNIQUE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Accomodation (
            Accomodation_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Accomodation_Type TEXT UNIQUE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS FactInsurance (
            Insurance_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Upper_Age INTEGER,
            Lower_Age INTEGER,
            Reco_Policy_Premium REAL,
            Holding_Policy_Duration REAL,
            Holding_Policy_Type TEXT,
            Is_Spouse TEXT,
            Reco_Insurance_Type TEXT,
            City_ID INTEGER,
            Accomodation_ID INTEGER,
            Indicator_ID INTEGER,
            class INTEGER,
            FOREIGN KEY (City_ID) REFERENCES City(City_ID),
            FOREIGN KEY (Accomodation_ID) REFERENCES Accomodation(Accomodation_ID),
            FOREIGN KEY (Indicator_ID) REFERENCES HealthIndicator(Indicator_ID)
        );
        """
    ]
    for sql in tables_sql:
        create_table(conn, sql)

# Step 3: Insert Data into the Database
def insert_into_database(data_dict, conn):
    # Insert into dimension tables
    city_values = [(city,) for city in set(data_dict["City_Code"])]
    insert_data(conn, "City", ["City_Code"], city_values)
    
    accomodation_values = [(acc,) for acc in set(data_dict["Accomodation_Type"])]
    insert_data(conn, "Accomodation", ["Accomodation_Type"], accomodation_values)
    
    health_values = [(health,) for health in set(data_dict["Health_Indicator"])]
    insert_data(conn, "HealthIndicator", ["Health_Indicator"], health_values)

    # Fetch primary keys for foreign keys
    city_mapping = {row[1]: row[0] for row in execute_sql_statement("SELECT City_ID, City_Code FROM City", conn)}
    accomodation_mapping = {row[1]: row[0] for row in execute_sql_statement("SELECT Accomodation_ID, Accomodation_Type FROM Accomodation", conn)}
    health_mapping = {row[1]: row[0] for row in execute_sql_statement("SELECT Indicator_ID, Health_Indicator FROM HealthIndicator", conn)}

    # Insert into fact table
    fact_values = [
        (
            int(data_dict["Upper_Age"][i].strip()),
            int(data_dict["Lower_Age"][i].strip()),
            float(data_dict["Reco_Policy_Premium"][i]),
            float(data_dict["Holding_Policy_Duration"][i].strip('+')),
            data_dict["Holding_Policy_Type"][i],
            data_dict["Is_Spouse"][i],
            data_dict["Reco_Insurance_Type"][i],
            city_mapping[data_dict["City_Code"][i]],
            accomodation_mapping[data_dict["Accomodation_Type"][i]],
            health_mapping[data_dict["Health_Indicator"][i]],
            int(data_dict["class"][i])
        )
        for i in range(len(data_dict["Upper_Age"]))
    ]
    insert_data(conn, "FactInsurance", 
                ["Upper_Age", "Lower_Age", "Reco_Policy_Premium", "Holding_Policy_Duration", 
                 "Holding_Policy_Type", "Is_Spouse", "Reco_Insurance_Type", "City_ID", 
                 "Accomodation_ID", "Indicator_ID", "class"], fact_values)

# Step 4: Main Execution
def main():
    db_file = "insurance_data.db"
    dataset_file = "/Users/vamsisaigarapati/Documents/python/insurance_dataset"
    conn = create_connection(db_file, delete_db=True)
    
    data_dict = parse_dataset(dataset_file)
    # print(data_dict)
    create_tables(conn)
    insert_into_database(data_dict, conn)
    conn.close()

if __name__ == "__main__":
    main()
